<a href="https://colab.research.google.com/github/sebagit914/Challenge01/blob/master/11_5_RNC_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

Importar datos

El conjunto de datos que usaremos acá es la base de datos MNIST que está construida en Keras Estos datos contienen muchos dígitos escritos a mano del 0 al 9. La tarea es clasificar cada imagen con el dígito que corresponde. Nuevamente, esta es una tarea fácil para nosotros, pero una históricamente difícil para un computador.

In [ ]:
# Obtener datos: ya está dividido en conjuntos de entrenamiento y prueba
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(60000, 28, 28)

Escalar datos

Queremos escalar los datos entre 0 y 1, Esta es  una opción recurrente porque mejora la velocidad. 

Ya que los pixeles son valores numéricos entre 0 y 225, podemos dividir por el valor máximo para lograrlo. 

In [ ]:
# Escalar datos entre 0 y 1
X_train = X_train / 255
X_test = X_test / 255

Reformar datos

Ahora necesitamos reformar nuestros datos. Actualmente, cada imagen es de 28 pixeles x 28 pixeles. Ya que cada pixel tiene un valor de color, tenemos que hacer una reforma para que sea 28x28x1. Si tenemos valores RGB (rojo, verde y azul, por sus iniciales en inglés), querremos reformar a 28x28x3.

In [ ]:
# Reformar datos
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

Formato objetivo

También necesitamos convertir nuestro objetivo en una categoría.

Ya que nuestro objetivo es numérico (0,2,3, etc.), nuestro computador va a reconocer los dígitos más grandes como valores numéricos altos. Sin embargo, todo lo que hacemos en este ejercicio es clasificar cuál imagen es cada una. En este caso, un 9 no tendría que tener un valor más alto que 1. Clasificar un 8 como 7 no es mejor que clasificarlo como un 2. Los dos son incorrectos.

Por suerte, Keras nos hace fácil esta tarea al usar la clase to_categorical.

In [ ]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

¡Ahora podemos construir la RNC!

Paso 1: Definir la estructura de nuestra red

In [ ]:
# Paso 1: Definir nuestra estructura de red
# Guardar la forma de entrada
input_shape = X_train.shape[1:4]
input_shape

(28, 28, 1)

In [ ]:
# Modelo secuencial
model = Sequential()

In [ ]:
# Capa convolucional
model.add(Conv2D(filters = 8, # Cuantos filtros quieres usar
                 kernel_size = 3, # tamaño de cada filtro
                 input_shape = input_shape)) # ¿Cuál es la forma de tus características de entrada (definimos esto arriba)
# Pooling layer
model.add(MaxPooling2D(pool_size = 2)) # Tamaño de pooling
# Capa de aplanamiento
model.add(Flatten())
# Capa de salida
model.add(Dense(10, # Cuantas posibilidades de salida tenemos
                activation = 'softmax')) # ¿Qué función de activación estás usando?

Paso 2: Compilar el modelo

Ya que este es un problema de clasificación multiclase, usaremos la entropía cruzada categórica como función de pérdida. También podemos rastrear la precisión a través del tiempo al especificarlo en los parámetros métricos.

In [ ]:
# Paso 2: compilar
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [ ]:
# Paso 3: ajustar nuestro modelo
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test), 
                    epochs=20)

Epoch 1/20
1875/1875 [==============================] - 18s 9ms/step - loss: 0.3486 - acc: 0.8996 - val_loss: 0.2295 - val_acc: 0.9357
Epoch 2/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2074 - acc: 0.9406 - val_loss: 0.1715 - val_acc: 0.9530
Epoch 3/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1528 - acc: 0.9574 - val_loss: 0.1321 - val_acc: 0.9636
Epoch 4/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1218 - acc: 0.9653 - val_loss: 0.1125 - val_acc: 0.9671
Epoch 5/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1020 - acc: 0.9703 - val_loss: 0.1026 - val_acc: 0.9696
Epoch 6/20
1553/1875 [=======================>......] - ETA: 2s - loss: 0.0891 - acc: 0.9744

In [ ]:
# Visualizar la pérdida
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.legend();

In [ ]:
# Visualizar la precisión
plt.plot(history.history['acc'], label='Train Accuracy')
plt.plot(history.history['val_acc'], label='Test Accuracy')
plt.legend();

Podemos ver que después de solo 20 epochs, obtenemos una precisión de prueba de ¡casi 98%!